## Get price data from specific website
Use requests, Beautiful Soup, and Pandas packages to retrieve html data from https://portland.hellodiem.com/, to locate product information, parse title and price into lists, remove superfluous pre-sale values, combine into a Pandas DataFrame, and save to a csv file.

In [ ]:
def get_heading_list(url, page):
    # Packages the request, send the request and catch the response: r
    r = requests.get(url+'page/'+str(page)+'/')
    html_text = r.text

    #set soup list equal to html text from page
    soup = BeautifulSoup(html_text, 'html.parser')
    
    headings = [[], []]
    #get list of product title headings on page
    headings[0] = soup.find_all('h2', class_='woocommerce-loop-product__title')
    
    #get list of product price headings on page
    headings[1] = soup.find_all('span', class_='woocommerce-Price-amount amount')
    pop_extra_price(headings[1])
    return headings

def pop_extra_price(headingss):
    """helper function to remove extra pre-sale prices from final list. Input list of 
    headers, get back list of headers less those that had a particular tag"""
    for item in headingss:
        if item.parent.name == 'del':
            headingss.pop(headingss.index(item))


In [2]:
def strip_product(header_list):
    """input a list of tag-type values and return list of strings with surrounding html characters removed"""
    string_list = ['' for item in range(len(header_list))]
    for item in range(len(header_list)):
        string_list[item] = str(header_list[item])[44:-5]
    return string_list

In [3]:
def strip_price(header_list):
    """input a list of tag-type values and return list of strings with surrounding html characters removed"""
    string_list = ['' for item in range(len(header_list))]
    for item in range(len(header_list)):
        string_list[item] = str(header_list[item])[101:-7]
    return string_list

In [4]:
from bs4 import BeautifulSoup
import requests
max_page = 12
url = 'https://portland.hellodiem.com/'

In [5]:
#create a list to hold lists of headers, one for each page
h_product = ['' for strng in range(max_page)]
h_price = ['' for strng in range(max_page)]

#iterate over each list of headers for a page, set each index of headers equal to list
for page in range(0, max_page):
    headers = get_heading_list(url, page+1)
    h_product[page] = headers[0]
    h_price[page] = headers[1]

In [6]:
combined_products = []
combined_prices = []

for sublist in range(max_page):
    combined_products.append(strip_product(h_product[sublist]))
    
for sublist in range(max_page):
    combined_prices.append(strip_price(h_price[sublist]))

In [21]:
#create a list of all prices
all_prices = []
for by_page in combined_prices:
    for li in by_page:
        #print(li, '\n')
        all_prices.append(float(li))
        
#create a list of all products
all_products = []
for by_page in combined_products:
    for li in by_page:
        #print(li, '\n')
        all_products.append(li)

In [24]:
import pandas as pd
product_ser = pd.Series(all_products)
price_ser = pd.Series(all_prices)
pot = pd.DataFrame([all_products, all_prices]).T


In [25]:
#pot.to_csv('prices05082018.csv')


In [32]:
pot[1].mean()

38.338862559241704